In [1]:
import pandas as pd # The only external library we will be using here.
import datetime # will be used later for formating date

## Merging of states data

* Aim:

    To merge all the states data into one single dataframe in the format:  
    ItemName | StateName | Jan 2016 | Feb 2016 | ...
  

* Steps:

 1. get the names of all the sheets
 2. check how the sheet looks
 3. create an empty data frame
 4. Loop through the sheet
 5. remove unwanted rows
 6. set the column names
 7. remove unwanted columns
 8. create a new column `State` with the sheet name as its value
 9. Concatenate the ouput of step 4 with the step 3 dataframe
 

## 2016 Data

In [2]:
workbook2016 = pd.read_excel('.\\Raw Data\\Selected food price watch mar 2017- proshare.xlsx', sheet_name=None) #read all sheets
workbook2016.keys() #check all the sheet names

odict_keys(['NATIONAL', 'ABIA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA', 'CROSS RIVER', 'DELTA', 'EDO', 'RIVERS', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'TARABA', 'YOBE', 'ABUJA', 'BENUE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'OGUN', 'ONDO', 'OSUN', 'OYO', 'JIGAWA', 'KADUNA', 'KANO', 'KATSINA', 'KEBBI', 'ZAMFARA', 'SOKOTO', 'Sheet1', 'Sheet2'])

In [3]:
to_delete = {'Sheet1', 'Sheet2'} #unwanted sheets
for k in to_delete:
    workbook2016.pop(k, None)

In [4]:
# Before proceed lets look at how one the sheets looks like,
workbook2016['NATIONAL']

,SELECTED FOOD PRICES WATCH,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,AVERAGE PRICES REPORTEDLY PAID BY HOUSEHOLDS A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year on Year %,Month on Month %
1,ItemLabels,Unit of Measurement,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,(Mar 2016 & Mar 2017),(Feb 2017 & Mar 2017)
2,Agric eggs medium size,1 Dozen,328.845,338.797,350.918,367.748,370.007,380.034,374.554,378.73,435.202,472.769,430.731,439.866,512.988,520.157,527.694,50.3756,1.44906
3,Agric eggs(medium size price of one),1 Piece,29.7483,29.9964,31.2013,35.9029,33.9841,34.1981,33.7892,33.9937,37.2044,42.0277,39.0496,39.4001,47.4181,42.9037,43.93,40.7956,2.39201
4,"Beans brown,sold loose",1Kg,236.808,238.021,240.914,263.575,256.413,276.594,288.324,293.56,305.292,345.482,357.265,347.604,353.604,337.111,353.277,46.6401,4.79557
5,Beans:white black eye. sold loose,1Kg,206.052,208.365,217.643,229.322,228.203,256.686,260.733,260.909,282.699,295.85,299.894,296.985,305.531,309.851,318.454,46.3192,2.77635
6,Beef Bone in,1Kg,811.242,790.983,826.956,884.993,865.164,851.814,884.058,910.405,953.914,979.606,965.587,948.071,1001.24,995.648,1010.29,22.1698,1.47061
7,"Beef,boneless",1Kg,977.328,1036.45,1004.41,1022.57,1047.33,1031.23,1055.21,1060.67,1145.29,1218.62,1153.03,1162.42,1249.48,1270.67,1281.71,27.6073,0.868397
8,Bread sliced 500g,1 loaf,225.4,240.176,259.291,249.734,248.672,276.972,306.354,265.221,293.427,305.266,305.716,296.052,302.872,299.75,297.506,14.738,-0.748651
9,Bread unsliced 500g,1 loaf,208.649,270.651,225.974,222.264,222.505,275.376,299.903,245.205,274.567,264.292,298.058,272.974,270.251,264.86,262.331,16.0894,-0.954627


The following may be noted in the sheets:
1. The head, first row and the last row are unfortunate, we would get rid of them
2. The last five rows, are also not needed in 2016 data
3. The The "Unit of Measurement" of measurement will be droped as it will definately be repetitive across the state. Though, it may be useful to let the populace know about the basis of the price, but we will drop it now and return it later during our final merging process.

In [5]:
merged_prices1 = pd.DataFrame()
for name, sheet in workbook2016.items():
    rf_data = sheet.copy(); #getting copy of the real sheet
    try:
        rf_data.columns = rf_data.iloc[1] #setting the colum name
        rf_data.drop([0,1],inplace=True) #deleting the first two rows
        rf_data=rf_data[:-1] # eliminating the last one unwanted row
        rf_data= rf_data[rf_data.columns[0:-5]] #eliminating the last five unwanted columns
        
        rf_data.drop(["Unit of Measurement"], axis=1,inplace=True) #will be saved later
        rf_data.set_index([rf_data.columns[0]], inplace=True)
        rf_data=pd.concat([rf_data],keys=[name])
        merged_prices1 = pd.concat([merged_prices1,rf_data], sort=True)
        
        
        #print("SUCCESS: " + name)
    except:
        print("ERROR: "+ name)
        raise Exception

# creating a mapper for ensuring that only the datetime types get formated 
mapper = lambda x:x.strftime("%b %Y") if isinstance(x,datetime.datetime) else x
merged_prices1.columns = merged_prices1.columns.map(mapper)
merged_prices1

1                                              Jan 2016 Feb 2016 Mar 2016  \
         ItemLabels                                                         
NATIONAL Agric eggs medium size                 328.845  338.797  350.918   
         Agric eggs(medium size price of one)   29.7483  29.9964  31.2013   
         Beans brown,sold loose                 236.808  238.021  240.914   
         Beans:white black eye. sold loose      206.052  208.365  217.643   
         Beef Bone in                           811.242  790.983  826.956   
         Beef,boneless                          977.328  1036.45  1004.41   
         Bread sliced 500g                        225.4  240.176  259.291   
         Bread unsliced 500g                    208.649  270.651  225.974   
         Broken Rice (Ofada)                    263.264  277.301  289.415   
         Chicken Feet                           595.477  610.777  648.568   
         Chicken Wings                           683.73  822.227  695.504   
         Evaporated tinned milk carnation 170g  119.419  120.456  122.793   
         Evaporated tinned milk(peak), 170g     137.622  137.875  139.165   
         Frozen chicken                         1149.31  1209.98  1176.47   
         Gari white,sold loose                  132.464  135.463  144.042   
         Gari yellow,sold loose                 156.407  157.399  166.396   
         Mudfish (aro) fresh                    793.213  839.335  823.617   
         Mudfish : dried                        1332.64  1372.69  1383.18   
         Onion bulb                             202.408  195.279  190.804   
         Rice agric sold loose                  202.666  213.321  227.225   
         Rice local sold loose                  163.285  172.745   200.43   
         Rice Medium Grained                    200.014  201.716  214.904   
         Rice,imported high quality sold loose  239.077  244.248  270.141   
         Tomato                                 223.497  209.162  207.238   
         Yam tuber                              147.709  149.769  156.586   
ABIA     Agric eggs medium size                  333.81  333.571  343.957   
         Agric eggs(medium size price of one)   29.4643  29.8214       30   
         Beans brown,sold loose                 281.151  288.889  289.316   
         Beans:white black eye. sold loose      264.881  265.575  273.771   
         Beef Bone in                           796.561  798.405  757.391   
...                                                 ...      ...      ...   
ZAMFARA  Rice local sold loose                  143.441  164.896  181.254   
         Rice Medium Grained                    133.938   185.86  148.955   
         Rice,imported high quality sold loose   216.87  229.861  253.156   
         Tomato                                 190.802  198.055  151.214   
         Yam tuber                              171.803  143.145  152.767   
SOKOTO   Agric eggs medium size                     360   362.34      360   
         Agric eggs(medium size price of one)        31  31.2015     31.5   
         Beans brown,sold loose                 182.695  183.883   198.72   
         Beans:white black eye. sold loose      136.161  137.046  153.981   
         Beef Bone in                           869.607   875.26      800   
         Beef,boneless                              975  981.337     1130   
         Bread sliced 500g                       218.75  220.172   256.25   
         Bread unsliced 500g                     206.25  207.591      250   
         Broken Rice (Ofada)                    148.852  149.819  163.795   
         Chicken Feet                           514.583      500  904.167   
         Chicken Wings                              650      650  666.667   
         Evaporated tinned milk carnation 170g      120   120.78    112.5   
         Evaporated tinned milk(peak), 170g      126.25  127.071      130   
         Frozen chicken                            1500  1509.75     1250

In [6]:
# Looks great!!! Let's give it a better name
merged_prices2016=merged_prices1

In [7]:
# If you need to see how it looks like in an excel window, or use it somewhere else
# uncomment the line below to get the output
#merged_prices2016.to_csv(".\\output\\merged_prices2016.csv") # Save as a CSV file

# 2017-Sep 2019 Data

In [8]:
workbook2017_2019 = pd.read_excel('.\\Raw Data\\SELECTED_FOOD SEP_2019.xlsx', sheet_name=None) #read all sheets
workbook2017_2019.keys() #check all the sheet names

odict_keys(['ABIA', 'ABUJA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA', 'CROSS RIVER', 'DELTA', 'EDO', 'RIVERS', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'TARABA', 'YOBE', 'BENUE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'ONDO', 'OGUN', 'OSUN', 'OYO', 'JIGAWA', 'KADUNA', 'KANO', 'KATSINA', 'KEBBI', 'ZAMFARA', 'SOKOTO', 'NATIONAL'])

**Since there is no un-wanted sheet, we can start merging the sheets**

In [9]:
# Before proceed lets look at how one the sheets looks like,
workbook2017_2019['ABIA']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year on Year %,Month on Month %
2,ItemLabels,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,...,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,(September 2018-September 2019),August 2019-September 2019
3,Agric eggs medium size,459.977,485.81,519.565,520.609,559.655,563.929,520.938,501.481,495.917,...,489.524,496.923,490.947,501.471,525.714,510,480.75,490,-0.29985,1.92408
4,Agric eggs(medium size price of one),45.1071,44.5136,46.4,48.1481,47.7419,48.4375,48.125,48.3333,46.9565,...,48.4615,49.3333,46.0909,47.5605,40,36.3667,42.35,48.3871,4.05827,14.2552
5,"Beans brown,sold loose",471.22,436.72,415.625,394.145,474.194,465.899,458.333,489.062,486.957,...,497.619,475,480.519,509.882,480.936,433.594,369.69,381.463,-23.7075,3.18445
6,Beans:white black eye. sold loose,420.764,375.09,400.242,391.872,470.312,466.749,463.793,484.375,468.571,...,486,462.143,464.422,472.46,453.438,420.714,389.68,385.979,-20.1423,-0.949796
7,Beef Bone in,1107.67,955.095,981.258,969.968,996.493,990.955,950.23,1050.81,996.219,...,898.166,903.455,976.538,988.643,992.451,1016.44,987.91,995.691,-3.18499,0.787611
8,"Beef,boneless",1378.4,1304.72,1129.16,1362.47,1499.91,1353.12,1356.8,1407.56,1481.51,...,1433.88,1404.94,1397.26,1404.3,1385.34,1350.86,1352.75,1373.66,-2.03049,1.5457
9,Bread sliced 500g,334.258,340.278,340,340.769,371.053,353.889,368.75,340,338.571,...,392.857,395.215,393.333,406.667,407.5,375.294,380,383.333,25,0.877193


The following may be noted in the sheets:
1. The head, first row and the last row are unfortunate, we would get rid of them
2. The last two rows are unwanted metrics, in this context.

In [10]:
merged_prices2 = pd.DataFrame()
for name, sheet in workbook2017_2019.items():
    rf_data = sheet.copy(); #getting copy of the real sheet
    try:
        rf_data.columns = rf_data.iloc[2] #setting the colum name
        rf_data.drop([0,1,2],inplace=True) #deleting the first row
        rf_data=rf_data[:-1] # eliminating the last unwanted row
        rf_data= rf_data[rf_data.columns[0:-2]] #eliminating the last two unwanted columns
        rf_data.set_index([rf_data.columns[0]], inplace=True)
        rf_data=pd.concat([rf_data],keys=[name])
        
        merged_prices2 = pd.concat([merged_prices2,rf_data], sort=True)                
        #print("SUCCESS: " + name)
    except:
        #print("ERROR: "+ name)
        raise Exception

# creating a mapper for ensuring that only the datetime types get formated 
mapper = lambda x:x.strftime("%b %Y") if isinstance(x,datetime.datetime) else x
merged_prices2.columns = merged_prices2.columns.map(mapper)
merged_prices2 #scoll down to see how it looks

Jan 2017 Feb 2017 Mar 2017  \
         ItemLabels                                                             
ABIA     Agric eggs medium size                     459.977   485.81  519.565   
         Agric eggs(medium size price of one)       45.1071  44.5136     46.4   
         Beans brown,sold loose                      471.22   436.72  415.625   
         Beans:white black eye. sold loose          420.764   375.09  400.242   
         Beef Bone in                               1107.67  955.095  981.258   
         Beef,boneless                               1378.4  1304.72  1129.16   
         Bread sliced 500g                          334.258  340.278      340   
         Bread unsliced 500g                        291.792  289.167  286.364   
         Broken Rice (Ofada)                            472      450  462.645   
         Chicken Feet                                774.76   731.25  735.714   
         Chicken Wings                               1060.1      900      920   
         Evaporated tinned milk carnation 170g       158.16      140      145   
         Evaporated tinned milk(peak), 170g         177.027      175    175.8   
         Frozen chicken                             1459.48  1509.72  1488.24   
         Gari white,sold loose                      270.932   267.46  286.957   
         Gari yellow,sold loose                     310.091  284.407  308.308   
         Mudfish (aro) fresh                         904.04  1015.22  1059.63   
         Mudfish : dried                            2642.63  2907.83  2984.74   
         Onion bulb                                 253.459  208.272  267.403   
         Rice agric sold loose                      440.704  393.008  386.209   
         Rice local sold loose                      363.704  382.846  366.182   
         Rice Medium Grained                         443.03  423.176  390.303   
         Rice,imported high quality sold loose       460.38  447.742  427.636   
         Tomato                                     309.656  306.309  333.333   
         Yam tuber                                  183.684  172.489  179.101   
         Dried Fish Sardine                          1759.8  1840.48  1524.31   
         Iced Sardine                                886.66   935.02  839.435   
         Irish potato                                249.69  293.965  299.075   
         Sweet potato                               116.168  116.008   112.68   
         Tilapia fish (epiya) fresh                  894.65  895.634  1052.63   
...                                                     ...      ...      ...   
NATIONAL Chicken Wings                               919.09  963.456   980.88   
         Dried Fish Sardine                         959.108   935.51  958.669   
         Evaporated tinned milk carnation 170g      136.288  140.526  143.564   
         Evaporated tinned milk(peak), 170g         157.746  166.143  177.054   
         Frozen chicken                             1419.75  1428.95  1555.51   
         Gari white,sold loose                      219.564  260.945  273.669   
         Gari yellow,sold loose                     255.825   250.45  302.047   
         Groundnut oil: 1 bottle, specify bottle     477.86   482.94    494.1   
         Iced Sardine                               1879.96  1915.68  1902.15   
         Irish potato                               300.946  307.291  311.805   
         Mackerel : frozen                          759.008  764.617  774.894   
         Maize grain white sold loose                167.11   172.86   174.03   
         Maize grain yellow sold loose               168.77   174.72   178.79   
         Mudfish (aro) fresh                        994.366  998.714  1079.16   
         Mudfish : dried                            1812.03   1955.1  2084.81   
         Onion bulb                                 258.916   241.44  246.949   
         Palm oil: 1 bottle,specify bottle           420.62   434.81   442.04   
 

### Ouch, we never expect those last two columns
This definately stems from the fact that the national columns contains more columns on the rigth of the
table which we don't need here. You can run ```workbook2["NATIONAL"]``` or check them through an excel window to confirm. but never edit the raw files. Its a lesson I have learnt the hard way.
> Never edit a raw data file to suit your needs, but you can make a copy, if neccessary.

The solution; I will solve this issue be simply cutting the last two rows off my dataframe.

In [11]:
merged_prices2017_2019=merged_prices2[merged_prices2.columns[0:-2]]
# Let's a look at only the first few rows
merged_prices2017_2019.head(5)

Jan 2017 Feb 2017 Mar 2017 Apr 2017  \
     ItemLabels                                                                 
ABIA Agric eggs medium size                459.977   485.81  519.565  520.609   
     Agric eggs(medium size price of one)  45.1071  44.5136     46.4  48.1481   
     Beans brown,sold loose                 471.22   436.72  415.625  394.145   
     Beans:white black eye. sold loose     420.764   375.09  400.242  391.872   
     Beef Bone in                          1107.67  955.095  981.258  969.968   

                                          May 2017 Jun 2017 Jul 2017 Aug 2017  \
     ItemLabels                                                                 
ABIA Agric eggs medium size                559.655  563.929  520.938  501.481   
     Agric eggs(medium size price of one)  47.7419  48.4375   48.125  48.3333   
     Beans brown,sold loose                474.194  465.899  458.333  489.062   
     Beans:white black eye. sold loose     470.312  466.749  463.793  484.375   
     Beef Bone in                          996.493  990.955   950.23  1050.81   

                                          Sep 2017 Oct 2017  ... Dec 2018  \
     ItemLabels                                              ...            
ABIA Agric eggs medium size                495.917  510.385  ...  495.789   
     Agric eggs(medium size price of one)  46.9565  47.7778  ...  49.0476   
     Beans brown,sold loose                486.957  471.346  ...    487.5   
     Beans:white black eye. sold loose     468.571  457.143  ...  480.159   
     Beef Bone in                          996.219  1046.75  ...  990.601   

                                          Jan 2019 Feb 2019 Mar 2019 Apr 2019  \
     ItemLabels                                                                 
ABIA Agric eggs medium size                  482.5  489.524  496.923  490.947   
     Agric eggs(medium size price of one)  46.3562  48.4615  49.3333  46.0909   
     Beans brown,sold loose                489.583  497.619      475  480.519   
     Beans:white black eye. sold loose      482.77      486  462.143  464.422   
     Beef Bone in                          987.675  898.166  903.455  976.538   

                                          May 2019 Jun 2019 Jul 2019 Aug 2019  \
     ItemLabels                                                                 
ABIA Agric eggs medium size                501.471  525.714      510   480.75   
     Agric eggs(medium size price of one)  47.5605       40  36.3667    42.35   
     Beans brown,sold loose                509.882  480.936  433.594   369.69   
     Beans:white black eye. sold loose      472.46  453.438  420.714   389.68   
     Beef Bone in                          988.643  992.451  1016.44   987.91   

                                          Sep 2019  
     ItemLabels                                     
ABIA Agric eggs medium size                    490  
     Agric eggs(medium size price of one)  48.3871  
     Beans brown,sold loose                381.463  
     Beans:white black eye. sold loose     385.979  
     Beef Bone in                          995.691  

[5 rows x 33 columns]

In [12]:
# If you need to see how it looks like in an excel window, or use it somewhere else
# uncomment the line below to get the output
#merged_prices2017_2019.to_csv(".\\output\\merged_prices2017_2019.csv") # Save as a CSV file

Now that we now have two dataframe, one for 2016 data and the other for 2017-2019 data What next?
1. We will merge them into one single datafram(Sheet)
2. Recall, that we dropped the unit of measurement columns in the 2016 data, though 2019-2019
data never contained it. Since we don't want the prices to look ambiguos, for example, the 
stated price of egg is for a crate of eggs, or a piece of egg?.  
Thus, we will make the unit accessible by concatenating the units with the name of the food Items, 
something like, *Rice Medium Grained / 1Kg*.
3. It is always a worthy additional optional step, when dealing with purely numeric values, with the intention that it will serve as input to another step or project, to cast the all the figures are numeric. I have seen values like 34.90 (float) written as '34.90' (string) before. I don't care about the cause, but I will always prevent such from my own end.

In [13]:
random_sheet = workbook2016['RIVERS']
random_sheet.columns = random_sheet.iloc[1] #setting the colum name
random_sheet.drop([0,1],inplace=True) #deleting the first two rows
random_sheet=random_sheet[:-1] # eliminating the last one unwanted row
unit_of_measure = random_sheet.iloc[:,0:2].set_index([random_sheet.columns[0]])
unit_of_measure = unit_of_measure.to_dict()['Unit of Measurement']
unit_of_measure # The Units of measure for each of the items.

{'Agric eggs medium size': '1 Dozen',
 'Agric eggs(medium size price of one)': '1 Piece',
 'Beans brown,sold loose': '1Kg',
 'Beans:white black eye. sold loose': '1Kg',
 'Beef Bone in': '1Kg',
 'Beef,boneless': '1Kg',
 'Bread sliced 500g': '1 loaf',
 'Bread unsliced 500g': '1 loaf',
 'Broken Rice (Ofada)': '1Kg',
 'Chicken Feet': '1Kg',
 'Chicken Wings': '1Kg',
 'Evaporated tinned milk carnation 170g': '1 unit',
 'Evaporated tinned milk(peak), 170g': '1 unit',
 'Frozen chicken': '1 unit',
 'Gari white,sold loose': '1Kg',
 'Gari yellow,sold loose': '1Kg',
 'Mudfish (aro) fresh': '1Kg',
 'Mudfish : dried': '1Kg',
 'Onion bulb': '1Kg',
 'Rice agric sold loose': '1Kg',
 'Rice local sold loose': '1Kg',
 'Rice Medium Grained': '1Kg',
 'Rice,imported high quality sold loose': '1Kg',
 'Tomato': '1Kg',
 'Yam tuber': '1Kg'}

In [14]:
# Final Steps
state_food_prices2016_2019 = merged_prices2016.join(merged_prices2017_2019, how="outer") #Merge
item_with_unit={} #dictionary of the ItemName/UnitOfMeasurement 
for key in unit_of_measure.keys():
     item_with_unit[key] = key + " / " + unit_of_measure[key]
state_food_prices2016_2019 = state_food_prices2016_2019.reset_index().rename(columns={"level_0":"States"})
state_food_prices2016_2019.replace({'ItemLabels':item_with_unit}, inplace=True)

# Making sure all figures are numerics
state_food_prices2016_2019 = state_food_prices2016_2019.apply(pd.to_numeric,downcast="float",errors='ignore')

# Let's see how it looks like
state_food_prices2016_2019

,States,ItemLabels,Jan 2016,Feb 2016,Mar 2016,Apr 2016,May 2016,Jun 2016,Jul 2016,Aug 2016,...,Dec 2018,Jan 2019,Feb 2019,Mar 2019,Apr 2019,May 2019,Jun 2019,Jul 2019,Aug 2019,Sep 2019
0,ABIA,Agric eggs medium size / 1 Dozen,333.809509,333.571442,343.956787,350.878784,355.588226,361.825409,361.944458,367.408905,...,495.789459,482.500000,489.523804,496.923065,490.947357,501.470764,525.714294,510.000000,480.750000,490.000000
1,ABIA,Agric eggs(medium size price of one) / 1 Piece,29.464285,29.821428,30.000000,31.500000,31.718750,33.333332,32.597221,32.658730,...,49.047619,46.356209,48.461540,49.333332,46.090908,47.560497,40.000000,36.366665,42.349998,48.387096
2,ABIA,"Beans brown,sold loose / 1Kg",281.150787,288.888885,289.316254,313.775360,303.928680,352.348999,368.750000,361.111115,...,487.500000,489.583344,497.619049,475.000000,480.519470,509.881989,480.935822,433.593750,369.690002,381.462585
3,ABIA,Beans:white black eye. sold loose / 1Kg,264.880951,265.575409,273.771362,297.735107,288.104095,332.222687,330.671387,350.198425,...,480.158722,482.770264,486.000000,462.142853,464.422089,472.460327,453.437500,420.714294,389.679993,385.978821
4,ABIA,Beef Bone in / 1Kg,796.560669,798.405457,757.390564,819.122559,832.112427,828.486511,835.154053,948.653076,...,990.601013,987.675171,898.166016,903.454895,976.537537,988.642517,992.450562,1016.440369,987.909973,995.690857
5,ABIA,"Beef,boneless / 1Kg",1029.080200,1158.491577,1119.505371,1117.487427,1226.228760,1190.488037,1237.830933,1239.606323,...,1460.281006,1421.607300,1433.878906,1404.936523,1397.257690,1404.302124,1385.337769,1350.863892,1352.750000,1373.659424
6,ABIA,Bread sliced 500g / 1 loaf,261.458344,250.555557,278.125000,268.402771,274.375000,281.250000,303.333344,312.857147,...,350.000000,372.000000,392.857147,395.214661,393.333344,406.666656,407.500000,375.294128,380.000000,383.333344
7,ABIA,Bread unsliced 500g / 1 loaf,241.388885,272.916656,247.083328,231.388885,235.625000,244.375000,245.000000,241.666672,...,261.538452,282.692322,305.000000,307.482544,326.666656,335.000000,322.500000,331.250000,332.119995,332.000000
8,ABIA,Broken Rice (Ofada) / 1Kg,350.339996,350.760406,351.181335,380.000000,360.000000,360.432007,360.864532,361.297546,...,419.253418,424.444458,424.783997,430.274384,427.272736,426.693878,415.032593,402.640015,405.055847,410.000000
9,ABIA,Catfish (obokun) fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1320.142334,1290.956543,1298.702271,1265.132446,1210.000000,1210.725952,1175.339966,1203.500000,1204.342407,1200.000000


In [15]:
#Looks as expected.
# Save as a CSV file
state_food_prices2016_2019.to_csv(".\\output\\state_food_prices2016_2019.csv", index=False)

### What Next
Sequel to this data wrangling operation, I have developed an [interactive data analysis 
web application](https://equimolar.com/dashboards/Nigeria_Food_Price_Trend) that consumes the CSV output. The [tool](https://equimolar.com/dashboards/Nigeria_Food_Price_Trend) was built with Pandas and Dash, a flask-based web framework.  

[Ewetoye, Ibrahim](https://EwetoyeIbrahim.github.io)